In [1]:
import sys, os

from streamlit.web.cli import config

# Add the project root (one level above 'research')
root_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(root_dir)


In [1]:
import os
from idlelib.pathbrowser import PathBrowser


In [2]:
%pwd


'C:\\Users\\91781\\PycharmProjects\\Kideney_Disease_Classification_MLflow_DVC\\research'

In [3]:
os.chdir(('../'))

In [4]:
%pwd

'C:\\Users\\91781\\PycharmProjects\\Kideney_Disease_Classification_MLflow_DVC'

## 🧩 What is “Data Ingestion”?

Data ingestion means:

Getting your dataset ready for the ML pipeline — by downloading, storing, and extracting it so the model can use it later.

Think of it as the “data collection + setup” stage.

# 🧱 Step 1: config.yaml — the settings file

This file tells your project where things are stored and where to get data from.

Example:

data_ingestion:
  root_dir: artifacts/data_ingestion
  source_url: https://drive.google.com/file/d/...
  local_data_file: artifacts/data_ingestion/data.zip
  unzip_dir: artifacts/data_ingestion


➡️ In simple terms:

root_dir: folder where all data-related files go.

source_url: where to download your dataset from.

local_data_file: where to save the downloaded zip file.

unzip_dir: where to extract the zip file

# ⚙️ Step 2: configuration.py — reads config.yaml

This file has a class like :
class ConfiguartionManager:
    def __init__(self):
        self.config = read_yaml(CONFIG_FILE_PATH)

    def get_data_ingestion_config(self):
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        return DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

➡️ What it does:

Reads your YAML file (so you don’t hardcode anything).

Creates necessary folders (like artifacts/data_ingestion/).

Prepares a DataIngestionConfig object — a simple container holding all paths and URLs for ingestion

# 📦 Step 3: data_ingestion.py — does the real work

This file contains a class, usually like:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        # Download dataset from Google Drive or any URL
        urllib.request.urlretrieve(self.config.source_url, self.config.local_data_file)

    def extract_zip_file(self):
        # Unzip the downloaded file
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(self.config.unzip_dir)

➡️ What it does in plain English:

Download the data from your given URL (Google Drive link).

Save it locally in the artifacts folder.

Extract the zip file so the raw dataset is available for further steps (like preprocessing).

# 🧠 Step 4: Try Block (Pipeline entry point)

Finally, in your notebook or main.py, you wrote something like:
try:
    config = ConfiguartionManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

➡️ What happens here:

Create the config manager.

Get all ingestion settings.

Create the ingestion object.

Download + unzip your dataset.

Your data is now ready for the next stage — e.g., data validation or preprocessing.

# ✅ End result:
After this step, your dataset is downloaded, extracted, and stored neatly under
artifacts/data_ingestion/

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)

class DataIngestionConfig:
    rootdir:Path
    source_url:str
    local_data_file:Path
    unzip_dir:Path

In [6]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories


In [15]:
class ConfiguartionManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):

        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config=self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            rootdir=config.root_dir,
            source_url=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [16]:
import os
import zipfile
import gdown
from src.cnnClassifier import  logger
from src.cnnClassifier.utils.common import get_size

In [19]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try:
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [20]:
try:
    config=ConfiguartionManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2025-10-28 17:15:49,047 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2025-10-28 17:15:49,050 : INFO : common : yaml file: params.yaml loaded successfully]
[2025-10-28 17:15:49,052 : INFO : common : created directory at: artifacts]
[2025-10-28 17:15:49,054 : INFO : common : created directory at: artifacts/data_ingestion]
[2025-10-28 17:15:49,056 : INFO : 3297092083 : Downloading data from https://drive.google.com/file/d/1zc-wA-zeQ_EEblpoJfOCX6HlOeomsbaW/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1zc-wA-zeQ_EEblpoJfOCX6HlOeomsbaW
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1zc-wA-zeQ_EEblpoJfOCX6HlOeomsbaW&confirm=t&uuid=9dea62bc-e3a7-4860-b5d9-37ad503e499c
To: C:\Users\91781\PycharmProjects\Kideney_Disease_Classification_MLflow_DVC\artifacts\data_ingestion\data.zip
100%|██████████| 940M/940M [03:36<00:00, 4.34MB/s] 

[2025-10-28 17:19:29,026 : INFO : 3297092083 : Downloaded data from https://drive.google.com/file/d/1zc-wA-zeQ_EEblpoJfOCX6HlOeomsbaW/view?usp=drive_link into file artifacts/data_ingestion/data.zip]
